In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install -U --q transformers datasets accelerate sentencepiece wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.6/281.6 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.9 MB/s eta 0:00:00


In [ ]:
%cd '/content/drive/MyDrive/NLU_NCKH/notebook/res_data_preprocesed'

/content/drive/.shortcut-targets-by-id/1btr8FZI1SJ1bZdaS-BqFfYlWxPaXfQOB/NLU_NCKH/notebook/res_data_preprocesed


# Import library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import torch
import datasets
from sklearn.metrics import *

# Load data

In [ ]:
df_train = pd.read_csv("train_res_preprocesed.csv")
df_dev = pd.read_csv("dev_res_preprocesed.csv")
df_test = pd.read_csv("test_res_preprocesed.csv")

print("Train: ", df_train.shape)
print("Dev: ",  df_dev.shape)
print("Test: ", df_test.shape )

Train:  (7028, 13)
Dev:  (771, 13)
Test:  (1938, 13)


# Preprocess data

In [ ]:
import numpy as np

def toTextLabel(row):
    dict_label = {1: "negative", 2: "neutral", 3: "positive"}
    labels = [
        f"{col.replace('&', '_').replace('#', '_')}-{dict_label[val]}"
        for col, val in zip(row.index, row.values) if val != 0
    ]
    return " , ".join(labels).lower()

df_train["Labels"] = df_train.drop("review", axis=1).apply(toTextLabel, axis=1)
df_test["Labels"] = df_test.drop("review", axis=1).apply(toTextLabel, axis=1)
df_dev["Labels"] = df_dev.drop("review", axis=1).apply(toTextLabel, axis=1)

In [ ]:
label_lengths = df_train["Labels"].map(len)
review_lengths = df_train["review"].map(len)

# Tìm độ dài lớn nhất và chỉ số của hàng có độ dài lớn nhất
max_review_length = review_lengths.max()
max_review_index = review_lengths.idxmax()

max_labels_length = label_lengths.max()
max_labels_index = label_lengths.idxmax()

print(f"Max length review: {max_review_length}")
print(f"Index of max review: {max_review_index}")
print(f"Value of Review at max index: {df_train['Labels'][max_review_index]}")
print(f"Value of Review at max index: {df_train['review'][max_review_index]}")
print("-"*80)
print(f"Max length labels: {max_labels_length}")
print(f"Index of max length: {max_labels_index}")
print(f"Value of Labels at max index: {df_train['Labels'][max_labels_index]}")
print(f"Value of Labels at max index: {df_train['review'][max_labels_index]}")

Max length review: 435
Index of max review: 5475
Value of Review at max index: food_quality-positive , food_style_options-positive , drinks_quality-positive , drinks_style_options-neutral
Value of Review at max index: toast lava kim_sa món này cũng ngon không kém bánh mới vừa mang ra nóng_hổi hết sảy con bà bảy luôn mới vừa dùng dao cắt ra là sốt kim_sa tràn ra mà thèm chảy nước_miếng luôn á và 1 ly trà xoài do uống li này một bữa ở bên đen đá pasteur thấy ngon quá nên từ đó thích món này luôn và bạn mình uống thấy cũng thích theo há há trà hoa_hồng add thêm trân châu_trắng uống cũng được có kem béo là thành ngon_hà rolling_on_the_floor_laughing
--------------------------------------------------------------------------------
Max length labels: 144
Index of max length: 6994
Value of Labels at max index: service_general-positive , restaurant_general-positive , restaurant_miscellaneous-positive , food_quality-positive , food_style_options-positive
Value of Labels at max index: bánh trà xa

In [ ]:
import random
import torch
from torch.utils.data import Dataset, DataLoader
# import pytorch_lightning as pl


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
from datasets import Dataset
dataset_train =  Dataset.from_pandas(df_train)
dataset_dev = Dataset.from_pandas(df_dev)
dataset_test = Dataset.from_pandas(df_test)

example = dataset_train[0]

print("Review:", example["review"])
print("Aspects + polarity:", example["Labels"])

Review: giá 53 k cỡ vừa
Aspects + polarity: drinks_prices-neutral , drinks_style_options-neutral


In [ ]:
dataset_train

Dataset({
    features: ['review', 'SERVICE#GENERAL', 'LOCATION#GENERAL', 'RESTAURANT#GENERAL', 'AMBIENCE#GENERAL', 'FOOD#PRICES', 'RESTAURANT#MISCELLANEOUS', 'DRINKS#PRICES', 'FOOD#QUALITY', 'FOOD#STYLE&OPTIONS', 'RESTAURANT#PRICES', 'DRINKS#QUALITY', 'DRINKS#STYLE&OPTIONS', 'Labels'],
    num_rows: 7028
})

# Prepare input for model

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import datasets

# Initialize constants and tokenizer
MAX_LEN_INPUT = 170
MAX_LEN_TARGET = 70
MODEL_NAME = "VietAI/vit5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_examples(examples):
    prefix = "aspect-based sentiment analysis: "
    reviews = [prefix + review for review in examples['review']]
    labels = [label for label in examples['Labels']]

    # Tokenize the reviews
    model_inputs = tokenizer(
        reviews,
        max_length=MAX_LEN_INPUT,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )

    # Tokenize the labels
    labels = tokenizer(
        labels,
        max_length=MAX_LEN_TARGET,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )

    # Adjust labels for compatibility with PyTorch
    temp_labels = labels['input_ids']
    temp_labels[temp_labels == tokenizer.pad_token_id] = -100

    # Add target_ids and target_mask to model_inputs
    model_inputs['labels'] = temp_labels
    model_inputs['decoder_attention_mask'] = labels['attention_mask']

    return model_inputs


# Apply the preprocessing function to the datasets
dataset_train = dataset_train.map(preprocess_examples, batched=True)
dataset_dev = dataset_dev.map(preprocess_examples, batched=True)
dataset_test = dataset_test.map(preprocess_examples, batched=True)

# Set the format of the datasets to PyTorch tensors
dataset_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])
dataset_dev.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])
dataset_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/7028 [00:00<?, ? examples/s]

Map:   0%|          | 0/771 [00:00<?, ? examples/s]

Map:   0%|          | 0/1938 [00:00<?, ? examples/s]

# Define metrics

In [ ]:
from sklearn.metrics import classification_report, precision_recall_fscore_support
true = ["drinks_prices-neutral, drinks_quality-positive",
         "drinks_prices-neutral , drinks_style_options-positive"]

pred = ["drinks_prices-positive, drinks_quality-positive",
         "drinks_prices-neutral , drinks_style_options-negative"]

aspects = df_train.drop(["review", "Labels"], axis=1).columns.tolist()
sentiments = {'negative': 1, 'neutral': 2, 'positive': 3}
replacements={0: 'None',
              1: 'negative',
              2: 'neutral',
              3: 'positive'}
def parse_labels(batch):
    outputs = []
    for batch in batch:
        y = [0] * len(aspects)
        for label in batch.split(","):
            aspect, sentiment = label.strip().rsplit("-", 1)
            appect = aspect.split('_', 1)
            aspect = f"{appect[0]}#{appect[1].replace('_', '&')}"
            y[aspects.index(aspect.upper())] = sentiments[sentiment]
        outputs.append(y)
    return outputs

def aspect_detection_eval(y_true, y_pred):
    aspect_test = []
    aspect_pred = []

    for row_test, row_pred in zip(y_true, y_pred):
        for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
            aspect_test.append(bool(col_test) * aspects[index])
            aspect_pred.append(bool(col_pred) * aspects[index])
    metrics = classification_report(aspect_test, aspect_pred, zero_division=1)
    return metrics

def sentiment_classification_eval(y_true, y_pred):
    y_true_flat = np.array(y_true).flatten()
    y_pred_flat = np.array(y_pred).flatten()
    target_names = list(map(str, replacements.values()))
    metrics = classification_report(y_true_flat, y_pred_flat, zero_division=1, target_names=target_names)
    return metrics

def combination_eval(y_test, y_pred):
    aspect_polarity_true = []
    aspect_polarity_pred = []

    for row_test, row_pred in zip(y_true, y_pred):
        for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
            aspect_polarity_true.append(f'{aspects[index]},{replacements[col_test]}')
            aspect_polarity_pred.append(f'{aspects[index]},{replacements[col_pred]}')
    metrics = classification_report(aspect_polarity_true, aspect_polarity_pred, zero_division=1)
    return metrics

def compute_metrics(outputs):
    preds = outputs.predictions
    labels = outputs.labels_ids

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    y_true = parse_labels(decoded_labels)
    y_preds = parse_labels(decoded_preds)

    return {
        "aspects metrics" : aspect_detection_eval(y_true, y_preds),
        "sentiment metrics" : sentiment_classification_eval(y_true, y_preds),
        "combination metrics": combination_eval(y_true, y_preds)
    }
# y_true = parse_labels(true)
# y_pred = parse_labels(pred)

# print("## Aspect Detection Evaluate ##")
# precision, recall, f1, _ = aspect_detection_eval(y_true, y_pred)
# print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")  # Formatted output

# print("\n## Sentiment Classification Evaluate ##")
# precision, recall, f1, _ = sentiment_classification_eval(y_true, y_pred)
# print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

# print("\n## Combination Evaluate (Aspect + Polarity detection) ##")
# precision, recall, f1, _ = combination_eval(y_true, y_pred)
# print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print('Device:', device)
!nvidia-smi

Device: cuda
Wed May 22 08:45:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+------------

# Training model

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
# model.to(device)
# Define training arguments
training_args = TrainingArguments(
    output_dir='./vit5-absa-e4-b4-v4-colab',
    num_train_epochs=4,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    # per_device_eval_batch_size=1,
    weight_decay=0.01,
    warmup_steps=500,
    eval_strategy="epoch",
    save_strategy='no'
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    # eval_dataset=dataset_dev,
    # compute_metrics=compute_metrics
)

# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: quy123456qqq (quidang). Use `wandb login --relogin` to force relogin


Step,Training Loss
500,1.810600
1000,0.187400
1500,0.140700
2000,0.115700
2500,0.094000


TrainOutput(global_step=2637, training_loss=0.44993385022727206, metrics={'train_runtime': 1359.8023, 'train_samples_per_second': 15.505, 'train_steps_per_second': 1.939, 'total_flos': 4263038257766400.0, 'train_loss': 0.44993385022727206, 'epoch': 3.0})

# Load model

In [ ]:
save = "/content/drive/MyDrive/NLU_NCKH/notebook/model/vit5-absa"
trainer.save_model(save)

In [ ]:
import gc
del model, tokenizer, trainer
torch.cuda.empty_cache()
gc.collect()

68

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "VietAI/vit5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(save)
model.to(device)
model

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

# Prepare input to inference

In [ ]:
class customDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.labels)

# Inference and Evaluate

In [ ]:
from torch.utils.data import DataLoader

# Evaluate the model
# results = trainer.evaluate()
# Generate predictions
MAX_LEN_INPUT = 170
MAX_LEN_TARGET = 70

def generate_predictions(dataset):
    prefix = "aspect-based sentiment analysis: "
    input_texts = [prefix + review for review in dataset['review']]
    y_test = dataset['Labels']

    inputs = tokenizer(
        input_texts,
        max_length=MAX_LEN_INPUT,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )
    encoding_data = customDataset(inputs, y_test)
    data_loader = DataLoader(encoding_data, batch_size=32, shuffle=False, num_workers=0)
    y_preds = []
    for index, data in enumerate(data_loader):
      outputs = model.generate(
          input_ids=data['input_ids'].to(device),
          attention_mask=data['attention_mask'].to(device),
          max_length=MAX_LEN_TARGET,
          num_beams=4,
          early_stopping=True
      )
      generate = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
      y_preds.extend(generate)

    return y_test, y_preds

dataset_test = datasets.Dataset.from_pandas(df_test)

y_test, y_preds = generate_predictions(dataset_test)

In [ ]:
y_true = parse_labels(y_test)
y_pred = parse_labels(y_preds)

print("## Aspect Detection Evaluate ##")
report = aspect_detection_eval(y_true, y_pred)
print(report)

print("\n## Sentiment Classification Evaluate ##")
report = sentiment_classification_eval(y_true, y_pred)
print(report)

print("\n## Combination Evaluate (Aspect + Polarity detection) ##")
report = combination_eval(y_true, y_pred)
print(report)

## Aspect Detection Evaluate ##
                          precision    recall  f1-score   support

                               0.98      0.98      0.98     20627
        AMBIENCE#GENERAL       0.91      0.82      0.87       227
           DRINKS#PRICES       0.82      0.79      0.80        47
          DRINKS#QUALITY       0.87      0.84      0.85       203
    DRINKS#STYLE&OPTIONS       0.76      0.63      0.69       129
             FOOD#PRICES       0.80      0.84      0.82       112
            FOOD#QUALITY       0.88      0.89      0.89       554
      FOOD#STYLE&OPTIONS       0.78      0.81      0.79       437
        LOCATION#GENERAL       0.93      0.92      0.93       104
      RESTAURANT#GENERAL       0.72      0.73      0.72       251
RESTAURANT#MISCELLANEOUS       0.68      0.66      0.67       145
       RESTAURANT#PRICES       0.89      0.69      0.78       117
         SERVICE#GENERAL       0.93      0.94      0.93       303

                accuracy                  